In [ ]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
final_merged_reviews = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/final_merged_reviews.csv")

In [ ]:
ratingColName = 'customer_sentiment'
final_merged_reviews[ratingColName] = 'POSITIVE'
for i in final_merged_reviews.index:
    if final_merged_reviews['rating'][i] > 3:
      final_merged_reviews[ratingColName][i] = 'POSITIVE'
    else:
      final_merged_reviews[ratingColName][i] = 'NEGATIVE'

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
barclays_stop_list = ['app', 'barclays','need','will']
stop_words.extend(barclays_stop_list)
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
nlp.Defaults.stop_words.update(barclays_stop_list)
nlp.Defaults.stop_words.update(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
len(nlp.Defaults.stop_words)

351

In [ ]:
final_merged_reviews_not_null = final_merged_reviews[final_merged_reviews['review_description'].notnull() & final_merged_reviews[ratingColName].str.contains('NEGATIVE')]
replace_dict = {"google pay":"google","contactless service":"contactless","credit card":"credit","debit card":"debit"}
final_merged_reviews_not_null['review_description'].replace(replace_dict,inplace=True,regex=True)
#work , pay -cheque = cheque , ok, calender view = calender 
#final_merged_reviews_not_null[final_merged_reviews_not_null["review_description"].str.contains("google")][["review_description", "rating",ratingColName]] 

In [ ]:
final_merged_reviews_not_null[final_merged_reviews_not_null["review_description"].str.contains("contactles")][["review_description", "rating",ratingColName]].shape

(726, 3)

In [ ]:

import re
wnl = WordNetLemmatizer()
def tokenizer(text):
    regex_num_ponctuation = '(\d+)|([^\w\s])'
    regex_little_words = r'(\b\w{1,2}\b)'
    return [wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct ]

def LemmaTokenizer(doc):
      wnl = WordNetLemmatizer()
      regex_num_ponctuation = '(\d+)|([^\w\s])'
      regex_little_words = r'(\b\w{1,2}\b)'
      return [wnl.lemmatize(t) for t in word_tokenize(doc) 
              if not re.search(regex_num_ponctuation, t) and not re.search(regex_little_words, t) and not t.is_stop and not t.is_punct]

In [ ]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
import lda
# create document term matrix
min_df = 4 # minimum required occurences of a word, e.g 4
max_features = 10000 # max number of unique words, e.g 10000

vectorizer = CountVectorizer(
              min_df=min_df, 
              max_features=max_features, 
              tokenizer=tokenizer, # use tokenizer we defined before
              ngram_range=(1,2)) # we use unigram & bigram
X = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=0481132da1668f0b0ff5c2a2a687c05d33af8b010a6eb55816aa5fc25e26f87b
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=ba62f6b7af2d21247041af3d723857467ead29ba8fd2f048847bcbadf0893595
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
from sklearn.decomposition import TruncatedSVD
import umap
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10, max_features=10000, tokenizer=tokenizer, ngram_range=(1, 2))
vz = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])



/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: ignored

In [ ]:
vectorizer

TfidfVectorizer(max_features=10000, min_df=10, ngram_range=(1, 2),
                tokenizer=<function tokenizer at 0x7f858a2c64d0>)

In [ ]:
svd_model = TruncatedSVD(n_components=50)
svd_tfidf = svd_model.fit_transform(vz)

umap_model = umap.UMAP(n_components=2)
umap_tfidf = umap_model.fit_transform(svd_tfidf)

In [ ]:
from sklearn.cluster import DBSCAN
eps = 2.5 # how close points should be to each other to be considered a part of a cluster, e.g 2.5
min_samples = 15 # the minimum number of points to form a dense region, e.g 15
dbscan = DBSCAN(
eps=eps,
min_samples=min_samples
)
dbscan_model = dbscan.fit(umap_tfidf)

In [ ]:
clusters = dbscan.fit_predict(umap_tfidf)

In [ ]:
df = pd.DataFrame()
df['dbscan_cluster'] = clusters
df['dbscan_cluster'] .value_counts()

 0    5335
 1     294
-1      29
 3      27
 4      24
 2      22
 5      15
Name: dbscan_cluster, dtype: int64

In [ ]:
# build LDA model
n_topics = 5 # pick the number of topics, e.g 5
n_iter = 2000 # number of learning iterations, e.g 2000

lda_model = lda.LDA(
              n_topics=n_topics, 
              n_iter=n_iter)
X_topics = lda_model.fit_transform(X) # X is document term matrix


# Print the topics found by the LDA model
n_top_words = 10
topic_summaries = []

topic_word = lda_model.topic_word_  # get the topic words


In [ ]:
vocab = vectorizer.get_feature_names_out()
vocab

array(['\n', '\n\n', ' ', ..., '🤦', '🤬', '🤷'], dtype=object)

In [ ]:
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(100+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: account bank money payment banking pay transaction slow like transfer update find change branch close time thing statement way payee app online balance new take make need barclaycard user add screen 2 check month feature set great detail long able year well show £ access try annoying local send delete go credicard simple notification option 
 people ok 

 difficult business remove switch issue security day open load bill log far useful list impossible card week easy basic page manage disappear direct reference leave review customer poor start have debit online banking date frustrating transfer money order allow system cash local branch terrible
Topic 1: pay payment contactless work cheque google contactless payment card google pay time bank mobile pay cheque log phone photo wallet support default update allow take fail change unreliable let google wallet like try rubbish bad have will constantly frustrating mobile payment try pay poor great ok picture shop ask set payment work

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 81.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatibl

In [ ]:
import pyLDAvis
def prepareLDAData(lda_model, lda_df, vectorizer):
    data = {
        'vocab': vocab,
        'doc_topic_dists': lda_model.doc_topic_,
        'doc_lengths': list(lda_df['review_description']),
        'term_frequency':vectorizer.vocabulary_,
        'topic_term_dists': lda_model.components_
    } 
    return data



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(len(lda_model.doc_topic_))
len(final_merged_reviews_not_null)


5746


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


5746

In [ ]:

pyLDAvis.enable_notebook()
panel = pyLDAvis.prepare(lda_model, vectorizer, X, vocab,mds='tsne')
panel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: ignored

In [ ]:
  lda_data = prepareLDAData(lda_model=lda_model, lda_df=X, vectorizer=vectorizer)
  prepared_data = pyLDAvis.prepare(**lda_data)
  pyLDAvis.display(prepared_data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: ignored

In [ ]:
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
# vis
def topic_modelling(sentiment,n_topics,n_top_words):
  final_merged_reviews_not_null_processed = final_merged_reviews[final_merged_reviews[ratingColName].str.contains(sentiment)]
  # create document term matrix
  min_df = 4 # minimum required occurences of a word, e.g 4
  max_features = 10000 # max number of unique words, e.g 10000

  vectorizer = CountVectorizer(
                
                min_df=min_df, 
                max_features=max_features, 
                tokenizer=tokenizer, # use tokenizer we defined before
                ngram_range=(1,2)) # we use unigram & bigram
  X = vectorizer.fit_transform(final_merged_reviews_not_null_processed['review_description'])
  # build LDA model
  n_topics = n_topics # pick the number of topics, e.g 5
  n_iter = 2000 # number of learning iterations, e.g 2000

  lda_model = lda.LDA(
                n_topics=n_topics, 
                n_iter=n_iter)
  X_topics = lda_model.fit_transform(X) # X is document term matrix


  # Print the topics found by the LDA model
  n_top_words = n_top_words
  topic_summaries = []

  topic_word = lda_model.topic_word_  # get the topic words
  vocab = vectorizer.get_feature_names_out()
  for i, topic_dist in enumerate(topic_word):
      topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
      topic_summaries.append(' '.join(topic_words))
      print('Topic {}: {}'.format(i, ' '.join(topic_words)))

  # load the pre-prepared pyLDAvis data
  lda_data = prepareLDAData(lda_model=lda_model, lda_df=topic_word, vectorizer=vectorizer)
  prepared_data = pyLDAvis.prepare(**lda_data)
  pyLDAvis.display(prepared_data)

In [ ]:
topic_modelling('NEGATIVE',5,10)

Topic 0: account card payment need banking transaction bank like change payee
Topic 1: update crash work keep time phone open log fix try
Topic 2: cheque pay take pay cheque work slow branch photo try time
Topic 3: account chat bank time service customer phone money help try
Topic 4: payment contactless work pay contactless payment google card google pay time mobile


NameError: ignored

In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatibl

In [ ]:
pd.set_option('display.max_colwidth', -1)

final_merged_reviews= final_merged_reviews_not_null.replace({'review_description': replace_dict}, inplace=True)
final_merged_reviews[final_merged_reviews["review_description"].str.contains("google")][["review_description", "rating",ratingColName]] 

<ipython-input-37-cfc8463ea43e>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


TypeError: ignored

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.6 MB/s eta 0:00:00


In [ ]:
!pip install HDBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for HDBSCAN: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3541543 sha256=0756dbdce681bc36e9d0e2dd4545ffded84c4bf75b90cad7bec66ba128229a04
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
Successfully built HDBSCAN


In [ ]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
# import lda
# create document term matrix
min_df = 4 # minimum required occurences of a word, e.g 4
max_features = 10000 # max number of unique words, e.g 10000

# vectorizer_model = CountVectorizer(
#               min_df=min_df, 
#               max_features=max_features, 
#               tokenizer=tokenizer, # use tokenizer we defined before
#               ngram_range=(1,2),
#               stop_words=stop_words) # we use unigram & bigram
# X = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])




# # we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=21fdb2a87a47c6d8256dc009dd963883ad35e7d8c90b5952e2d1587947f10c3e
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=cc686400a4988433ead1d463bc21bb816011a1457bb9809497d5002935ddea04
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
final_merged_reviews_not_null.reset_index(drop=True, inplace=True)
from bertopic import BERTopic

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(final_merged_reviews_not_null['review_description'])

Batches:   0%|          | 0/180 [00:00<?, ?it/s]

2023-04-29 09:30:28,717 - BERTopic - Transformed documents to Embeddings
2023-04-29 09:30:38,086 - BERTopic - Reduced dimensionality
2023-04-29 09:30:39,154 - BERTopic - Clustered reduced embeddings


In [121]:
freq = model.get_topic_info()
freq

,Topic,Count,Name
0,-1,586,-1_mortgage_account_google_work
1,0,1759,0_contactless_payment_payments_pay
2,1,1093,1_pay_contactless_google_phone
3,2,947,2_update_open_crashes_phone
4,3,259,3_slow_difficult_useless_takes
5,4,237,4_cheque_cheques_photo_pay
6,5,199,5_chat_service_chat function_function
7,6,187,6_calendar_view_calendar view_calender
8,7,175,7_contactless_working_mobile_work
9,8,120,8_log_login_account_keeps


In [122]:
model.visualize_topics()

In [ ]:
len(topics)

5746

In [124]:
model.visualize_barchart(topics=[10])

In [126]:

topics_to_merge = [[0, 1],[0,7],[2,9]]
model.merge_topics(final_merged_reviews_not_null['review_description'], topics_to_merge)

In [127]:
model.visualize_barchart()

In [134]:
model.save('negative_sentiment_model') 

In [ ]:
negative_sentiment_model = BERTopic.load("negative_sentiment_model")

In [135]:
negative_sentiment_model.visualize_barchart()

In [129]:
negative_sentiment_model.visualize_barchart(topics=[-1])